In [13]:
from langgraph.graph import StateGraph, START, END
from utils.state import DocumentState
from utils.nodes import (
    decide_split,
    split_into_chunks,
    generate_xmindmark_direct,
    generate_xmindmark_for_chunk,
    merge_all_xmindmarks,
    generate_global_title_node,
)
from IPython.display import Image, display


def build_graph():
    builder = StateGraph(DocumentState)

    builder.add_node("check_split", decide_split)
    builder.add_node("generate_global_title", generate_global_title_node)
    builder.add_node("split_chunks", split_into_chunks)
    builder.add_node("generate_direct", generate_xmindmark_direct)
    builder.add_node("generate_chunk_xmind", generate_xmindmark_for_chunk)
    builder.add_node("merge_xmind", merge_all_xmindmarks)

    builder.add_edge(START, "check_split")

    # Điều hướng theo `need_split`
    builder.add_conditional_edges(
        "check_split",
        lambda state: "split_chunks" if state["need_split"] else "generate_direct", path_map={"split_chunks": "split_chunks", "generate_direct": "generate_direct"}
    )

    # Sau khi split
    builder.add_edge("split_chunks", "generate_chunk_xmind")

    # Loop qua từng chunk
    def should_continue(state: DocumentState):
        return (
            "generate_chunk_xmind"
            if len(state["xmindmark_chunks_content"]) < len(state["chunks"])
            else "generate_global_title"
        )

    builder.add_conditional_edges("generate_chunk_xmind", should_continue, {"generate_chunk_xmind": "generate_chunk_xmind", "generate_global_title": "generate_global_title"})

    builder.add_edge("generate_global_title", "merge_xmind")

    # Kết thúc
    builder.add_edge("merge_xmind", END)
    builder.add_edge("generate_direct", END)

    return builder.compile()


graph = build_graph()

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

ImportError: cannot import name 'generate_global_title_node' from 'utils.nodes' (/home/misa/work/xmind2/agent/utils/nodes.py)